In [2]:
import pandas as pd
import math
import plotly.express as px

# Cleaning Sample History

In [63]:
df1 = pd.read_csv("Sample-History1.csv")
df2 = pd.read_csv("Sample-History3.csv")
#df = df.rename(columns = {"Title": "History"})

In [ ]:
df2

In [ ]:
# df['Date'] = pd.to_datetime(df['Date'])
# df['Day']= df['Date'].dt.day
# df['Month']= df['Date'].dt.month
# df['Year']= df['Date'].dt.year
# df['Day_of_week'] = df['Date'].dt.dayofweek # monday = 0, sunday = 6

In [ ]:
# df

In [ ]:
# df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
# df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
# df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]

In [ ]:
# df['Type'] = df['Episode'].apply(lambda x : 'Movie' if (pd.isna(x)==True) else 'TV')

Right now, movies that have a colon in their title have an incomplete value in the `Title`. I correct that below:

In [ ]:
# tv = df[df['Type']!='Movie']
# movies = df[df['Type']=='Movie']
# movies['Title'] = movies['History']
# movies['Season'] = None

In [ ]:
# tv.head()

In [ ]:
# tv['Season'] = tv['Season'].str.split().str[1]

In [ ]:
# df = pd.concat([movies, tv], ignore_index = True)

In [55]:
def clean_watch_history(df):
    '''
    Function that cleans a given users watch history data
    Input: dataframe
    Output: (cleaned) dataframe
    '''
    df = df.rename(columns = {"Title": "History"})
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day']= df['Date'].dt.day
    df['Month']= df['Date'].dt.month
    df['Year']= df['Date'].dt.year
    df['Day_of_week'] = df['Date'].dt.dayofweek

    df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
    df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
    df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]

    df['Type'] = df['Episode'].apply(lambda x : 'Movie' if (pd.isna(x)==True) else 'TV')

    tv = df[df['Type']!='Movie']
    tv['Season'] = tv['Season'].str.split().str[1]

    movies = df[df['Type']=='Movie']
    movies['Title'] = movies['History']
    movies['Season'] = None

    df = pd.concat([movies, tv], ignore_index = True)
    return df

In [64]:
df1 = clean_watch_history(df1)
df2 = clean_watch_history(df2)

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3536717928.py:14: FutureWarning:

In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3536717928.py:15: FutureWarning:

In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3536717928.py:16: FutureWarning:

In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3536717928.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [ ]:
df2[df2['History']=='Extraordinary Attorney Woo: The Blue Night of Jeju I']

Issues:

1. (**SOLVED**) TV shows that have a colon in their title need to be handled differently. See `Avatar: The Last Airbender`.
    - Should we consider only calling split() on ': Season', ': Series', ': Book', etc? An obvious issue is it would be hard to compile an exhaustive such list. Other examples include Volume, Chapter, etc.
    - Alternatively, we could keep a list of all TV shows from the Netflix dataset that do have a colon. If any show then appears in that list, we could then make sure its first colon is not counted in the above procedure. Again, an obvious issue is figuring out how to determine that a given show is in that list (since this has to occur before we do any cleaning)
    - A potential solution is to do split on colon occurences in reverse order. For instance, only split based on the last two occurences of the colon character, and leave earlier colon occurences alone.
    
2. Movie titles may not exactly match the Netflix Dataset. See `Major (Telugu)`, which appears in the Netflix dataset just as `Major`, but in 3 different rows with 3 different languages.
    - One solution would be to remove the `([language])` substring in the sample dataset, and drop duplicates in the Netflix dataset (since the only differences between the duplicates would be the language).
    - Another solution would be to instead fuzzy match based on string similarity. Theoretically, this should work since exact matches would just have a 100% score. Plus, if titles in the Netflix dataset have this issue too (i saw at least one occurence), this might be able to handle those edge cases. Based on past experience, can be pain to implement (that being said, the logic is simpler than what I've previously had to deal with).
        - We could also think about first doing exact string matching, and then performing fuzzy matching on the remaining titles that haven't been matched.
    - Another solution is to just ignore :) and hope it doesn't affect our recommendations much
3. TV Shows that have a colon in their episode name. See `New Girl: Season 6: Operation: Bobcat`.
    - Similar issue to 1, but flipped. Hard to solve both issues 1 and 3. Maybe just ignore?

# Merging with Netflix Data

In [57]:
def netflix_merge(df):
    '''
    Function that merges given watch history with netflix dataset,
    and returns merged dataset
    '''
    titles = pd.read_csv('titles.csv')
    merged = df.merge(titles, left_on = 'Title', right_on = 'title', how = 'inner')
    cols_to_drop = ['type', 'production_countries', 'imdb_id', 'age_certification', 
                    'title', 'seasons', 'tmdb_popularity']
    merged = merged.drop(cols_to_drop, axis = 1)
    return merged

In [65]:
df1 = netflix_merge(df1)
df2 = netflix_merge(df2)

In [ ]:
# titles = pd.read_csv("titles.csv")

In [ ]:
# merged = df.merge(titles, left_on = 'Title', right_on = 'title', how = 'inner')
# only 7 rows were dropped

In [ ]:
# merged.info()

In [ ]:
# cols_to_drop = ['type', 'production_countries', 'imdb_id', 'age_certification', 'id', 'title', 'seasons',
#                'tmdb_popularity']
# merged = merged.drop(cols_to_drop, axis = 1)

In [ ]:
# merged.head()

# Overlap Merge

In [66]:
def overlap_merge(df1, df2):
    dfNew = df1.merge(df2, how = 'inner', left_on = 'History', right_on = 'History', suffixes=('', '_y'))
    dfNew = dfNew.drop(dfNew.filter(regex='_y$').columns, axis=1)
    return dfNew

In [67]:
overlap = overlap_merge(df1, df2)

In [68]:
overlap

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type,id,description,release_year,runtime,genres,imdb_score,imdb_votes,tmdb_score
0,Avatar: The Last Airbender: Book 1: The Boy in...,2022-09-30,30,9,2022,4,Avatar: The Last Airbender,1,The Boy in the Iceberg,TV,ts3371,"In a war-torn world of elemental magic, a youn...",2005,24,"['scifi', 'animation', 'action', 'family', 'fa...",9.3,303666.0,8.70
1,"New Girl: Season 1: Fancyman, Part 1",2022-03-08,8,3,2022,1,New Girl,1,"Fancyman, Part 1",TV,ts20564,Jessica Day is an offbeat and adorable girl in...,2011,21,"['comedy', 'romance']",7.7,219079.0,7.30
2,New Girl: Season 1: Control,2022-03-08,8,3,2022,1,New Girl,1,Control,TV,ts20564,Jessica Day is an offbeat and adorable girl in...,2011,21,"['comedy', 'romance']",7.7,219079.0,7.30
3,New Girl: Season 1: Injured,2022-01-08,8,1,2022,5,New Girl,1,Injured,TV,ts20564,Jessica Day is an offbeat and adorable girl in...,2011,21,"['comedy', 'romance']",7.7,219079.0,7.30
4,New Girl: Season 1: Bully,2022-01-08,8,1,2022,5,New Girl,1,Bully,TV,ts20564,Jessica Day is an offbeat and adorable girl in...,2011,21,"['comedy', 'romance']",7.7,219079.0,7.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Breaking Bad: Season 1: Gray Matter,2022-03-16,16,3,2022,2,Breaking Bad,1,Gray Matter,TV,ts4,"When Walter White, a New Mexico chemistry teac...",2008,48,"['drama', 'crime', 'thriller']",9.5,1775990.0,8.79
151,Breaking Bad: Season 1: Cancer Man,2022-03-16,16,3,2022,2,Breaking Bad,1,Cancer Man,TV,ts4,"When Walter White, a New Mexico chemistry teac...",2008,48,"['drama', 'crime', 'thriller']",9.5,1775990.0,8.79
152,Breaking Bad: Season 1: And the Bag's in the R...,2022-03-16,16,3,2022,2,Breaking Bad,1,And the Bag's in the River,TV,ts4,"When Walter White, a New Mexico chemistry teac...",2008,48,"['drama', 'crime', 'thriller']",9.5,1775990.0,8.79
153,Breaking Bad: Season 1: The Cat's in the Bag,2022-03-15,15,3,2022,1,Breaking Bad,1,The Cat's in the Bag,TV,ts4,"When Walter White, a New Mexico chemistry teac...",2008,48,"['drama', 'crime', 'thriller']",9.5,1775990.0,8.79


# Aggregation

Stats/Visualizations to create:
- Individual
    - Viewing history by day of week
    - Top 10 Shows Watched
    - Movies most rewatched
    - Viewing history by month
    - Viewing history trend over time
    - Average watch time per day
- Common
    - Most common TV shows
    - Most common movies
    - Most common genres
    - Favorite actors/directors
    - Match score
- Overall
    - Minutes spent vs all other users
    - Percentile for top shows

**For all of these, we could use number of minutes watched (using the runtime column) rather than just the number of episodes**

1. Total Minutes Watched

In [ ]:
def total_minutes(df):
    fig = px.pie(df, values='runtime', names='Type', hole=.5)
    fig.update_traces(hovertemplate='Total Watch Time: %{value} mins')
    return fig

In [ ]:
total_minutes(df1).show()

In [ ]:
df = merged.groupby('Type').sum()

In [ ]:
fig = px.pie(merged, values='runtime', names='Type', hole=.5)
fig.update_traces(hovertemplate='Total Watch Time: %{value} mins')
fig.show()

2. Most watched TV shows

In [ ]:
def most_watched_tv(df):
    df = df[df['Type']=='TV']
    df = df.groupby('Title').sum()
    df = df.reset_index()
    
    df = df.sort_values(by='runtime', ascending= False)
    # extract 10 most similar shows
    df = df.head(10)
    
    fig = px.bar(data_frame= df,
             x= "runtime",
             y= "Title",
             # text= "num_shared_actors",
             labels= {"runtime": "Minutes Watched"},
             text_auto= True)
    fig.update_traces(hovertemplate='Total Watch Time: %{x} mins')
    return fig

In [ ]:
most_watched_tv(df1).show()

In [ ]:
df = merged[merged['Type']=='TV']
df = df.groupby('Title').sum()
df = df.reset_index()

In [ ]:
df = df.sort_values(by='runtime', ascending= False)
# extract 10 most similar shows
df = df.head(10)

In [ ]:
fig = px.bar(data_frame= df,
             x= "runtime",
             y= "Title",
             # text= "num_shared_actors",
             labels= {"runtime": "Minutes Watched"},
             text_auto= True)
fig.update_traces(hovertemplate='Total Watch Time: %{x} mins')
fig.show()

3. Most watched genre

In [ ]:
def most_watched_genre(df):
    df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

In [ ]:
df1

In [75]:
df = df1
df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

In [76]:
df = df['genres'].str.split(',', expand = True)

In [78]:
temp1 = df1['genres'].str.split(',', expand = True)
genre_col_list = ['genre_' + str(i) for i in range(len(temp1.columns))]
temp1.columns = genre_col_list

In [79]:
genre_col_list


['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4']

In [80]:
temp = pd.concat([df1, temp1], axis = 1)

In [81]:
s = pd.Series()
for i in genre_col_list:
    s = s.add(temp.groupby(i).sum()['runtime'], fill_value=0)

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/4104630834.py:1: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/4104630834.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/4104630834.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/4104630834.py:3: FutureWarning:

The default value

In [95]:
s.idxmax()

'drama'

In [96]:
pd.DataFrame(s.reset_index())

,index,0
0,action,291.0
1,animation,24.0
2,comedy,3399.0
3,crime,4998.0
4,drama,6041.0
5,family,24.0
6,fantasy,24.0
7,history,558.0
8,romance,3773.0
9,scifi,24.0


In [97]:
def most_watched_genre(df):
    df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

    temp = df['genres'].str.split(',', expand = True)
    genre_col_list = ['genre_' + str(i) for i in range(len(temp.columns))]
    temp.columns = genre_col_list

    df = pd.concat([df, temp], axis = 1)

    s = pd.Series()
    for i in genre_col_list:
        s = s.add(df.groupby(i).sum()['runtime'], fill_value=0)
    
    top_genre = s.idxmax()
    df = pd.DataFrame(s.reset_index())

    fig = px.pie(df, values=0, names='index', hole=.5)
    fig.update_traces(hovertemplate='Number of Minutes Watched: %{value}')
    return fig, top_genre

In [99]:
fig, top_genre = most_watched_genre(df1)
top_genre

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3544281500.py:13: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3544281500.py:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3544281500.py:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_10681/3544281500.py:15: FutureWarning:

The default v

'drama'

4. Top cast

In [28]:
cast = pd.read_csv("credits.csv")

In [32]:
cast = cast[cast['role']=='ACTOR']

In [34]:
temp = df1.merge(cast, how='left', left_on='id', right_on='id')

In [35]:
temp[temp['name']=='Poorna Jagannathan']

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type,...,release_year,runtime,genres,imdb_score,imdb_votes,tmdb_score,person_id,name,character,role
4,Delhi Belly,2022-04-08,8,4,2022,4,Delhi Belly,None,NaN,Movie,...,2011,102,"['comedy', 'action', 'crime', 'thriller']",7.5,29927.0,7.000,91185,Poorna Jagannathan,Menaka Vashisht,ACTOR
385,Never Have I Ever: Season 3: ...lived the dream,2022-12-08,8,12,2022,3,Never Have I Ever,3,...lived the dream,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
393,Never Have I Ever: Season 3: …had an Indian bo...,2022-12-08,8,12,2022,3,Never Have I Ever,3,…had an Indian boyfriend,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
401,Never Have I Ever: Season 3: ...hooked up with...,2022-12-08,8,12,2022,3,Never Have I Ever,3,...hooked up with my boyfriend,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
409,Never Have I Ever: Season 3: ...cheated,2022-12-08,8,12,2022,3,Never Have I Ever,3,...cheated,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
417,Never Have I Ever: Season 3: ...had a breakdown,2022-12-08,8,12,2022,3,Never Have I Ever,3,...had a breakdown,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
425,Never Have I Ever: Season 3: ...been ghosted,2022-12-08,8,12,2022,3,Never Have I Ever,3,...been ghosted,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
433,Never Have I Ever: Season 3: ...made someone j...,2022-12-08,8,12,2022,3,Never Have I Ever,3,...made someone jealous,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
441,Never Have I Ever: Season 3: …had a valentine,2022-12-08,8,12,2022,3,Never Have I Ever,3,…had a valentine,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR
449,Never Have I Ever: Season 1: Pilot,2022-11-08,8,11,2022,1,Never Have I Ever,1,Pilot,TV,...,2020,28,"['comedy', 'drama']",7.8,47000.0,8.183,91185,Poorna Jagannathan,Dr. Nalini Vishwakumar,ACTOR


In [46]:
temp = temp.groupby('name').count()['Title'].reset_index()

In [47]:
temp.head()

,name,Title
0,Aamani,1
1,Aamir Khan,1
2,Aaron Paul,53
3,Abhay Bethiganti,1
4,Adjoa Andoh,9


In [48]:
temp = temp.sort_values(by='Title', ascending= False)
# extract 10 most similar shows
temp = temp.head(10)
fig = px.bar(data_frame= temp,
             x= "Title",
             y= "name",
             # text= "num_shared_actors",
             labels= {"Title": "Number of Appearances"},
             text_auto= True)
fig.update_traces(hovertemplate='Total Appearances: %{x}')
fig.show()

In [69]:
def top_actors(df):
    cast = pd.read_csv("credits.csv")
    cast = cast[cast['role']=='ACTOR']
    df = df.merge(cast, how='left', left_on='id', right_on='id')

    df = df.groupby('name').count()['Title'].reset_index()
    df = df.sort_values(by='Title', ascending= False)
    df = df.head(10)

    fig = px.bar(data_frame= df,
                x= "Title",
                y= "name",
                # text= "num_shared_actors",
                labels= {"Title": "Number of Appearances"},
                text_auto= True)
    fig.show()

In [70]:
top_actors(overlap)

5. language breakdown

In [71]:
df1

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type,id,description,release_year,runtime,genres,imdb_score,imdb_votes,tmdb_score
0,Delhi Belly,2022-04-08,8,4,2022,4,Delhi Belly,None,NaN,Movie,tm177258,"Three unsuspecting, average guys find themselv...",2011,102,"['comedy', 'action', 'crime', 'thriller']",7.5,29927.0,7.000
1,Most Eligible Bachelor,2021-11-24,24,11,2021,2,Most Eligible Bachelor,None,NaN,Movie,tm876997,Akhil and Pooja have a love story with a touch...,2021,170,"['drama', 'romance']",6.1,3261.0,5.700
2,Better Call Saul: Season 5: Something Unforgiv...,2023-02-15,15,2,2023,2,Better Call Saul,5,Something Unforgivable,TV,ts8,Six years before Saul Goodman meets Walter Whi...,2015,49,"['drama', 'crime']",8.8,438575.0,8.500
3,Better Call Saul: Season 5: Bad Choice Road,2023-02-15,15,2,2023,2,Better Call Saul,5,Bad Choice Road,TV,ts8,Six years before Saul Goodman meets Walter Whi...,2015,49,"['drama', 'crime']",8.8,438575.0,8.500
4,Better Call Saul: Season 5: Bagman,2023-02-15,15,2,2023,2,Better Call Saul,5,Bagman,TV,ts8,Six years before Saul Goodman meets Walter Whi...,2015,49,"['drama', 'crime']",8.8,438575.0,8.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Breaking Bad: Season 1: The Cat's in the Bag,2022-03-15,15,3,2022,1,Breaking Bad,1,The Cat's in the Bag,TV,ts4,"When Walter White, a New Mexico chemistry teac...",2008,48,"['drama', 'crime', 'thriller']",9.5,1775990.0,8.790
266,Breaking Bad: Season 1: Pilot,2022-03-15,15,3,2022,1,Breaking Bad,1,Pilot,TV,ts4,"When Walter White, a New Mexico chemistry teac...",2008,48,"['drama', 'crime', 'thriller']",9.5,1775990.0,8.790
267,Squid Game: Season 1: One Lucky Day,2021-11-21,21,11,2021,6,Squid Game,1,One Lucky Day,TV,ts220305,Hundreds of cash-strapped players accept a str...,2021,55,"['action', 'thriller', 'drama']",8.0,426967.0,7.821
268,Squid Game: Season 1: VIPS,2021-11-18,18,11,2021,3,Squid Game,1,VIPS,TV,ts220305,Hundreds of cash-strapped players accept a str...,2021,55,"['action', 'thriller', 'drama']",8.0,426967.0,7.821
